# Part 1: Preprocessing and Common Methods

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error

# 1. Data Preparation
The methods of data cleaning and feature engineering are discussed in our accompanying report. The `data_processed.csv` file is used for subsequent questions.

In [ ]:
sales=pd.read_csv("https://raw.githubusercontent.com/hoganj15/MMA_Assignment_Data/main/Service/data_processed.csv") # you'll have to change the filepath here to get the data_preprocessed.csv file on your drive
sales

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_2,functionality_3,functionality_4,functionality_5,functionality_6,functionality_7,functionality_8,functionality_9,functionality_10,functionality_11,functionality_12,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,vendor_3,vendor_4,vendor_5,vendor_6,vendor_7,vendor_8,vendor_9,vendor_10
0,2016-11-14,1,110.0,10.24,9.86,10.16,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2016-11-21,1,127.0,8.27,10.24,9.86,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2016-11-28,1,84.0,8.83,8.27,10.24,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2016-12-05,1,87.0,8.98,8.83,8.27,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2016-12-12,1,64.0,10.40,8.98,8.83,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,2018-08-27,44,20.0,53.99,42.38,43.99,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4308,2018-09-03,44,14.0,52.99,53.99,42.38,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4309,2018-09-10,44,22.0,44.99,52.99,53.99,1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4310,2018-09-17,44,28.0,42.99,44.99,52.99,1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
skus = list(sales['sku'].unique()) #unique skus for decentralized approach

In [ ]:
#we're trying to predict weekly sales
data = dict()
colnames = [col for col in sales.columns if col not in ["week", "weekly_sales", "sku"]] #variables we don't want as predictors
for sku in skus:
  df_sku = sales[sales['sku'] == sku]
  data[sku] = {"X": df_sku[colnames].values, "y": df_sku["weekly_sales"].values}

In [ ]:
data[1]["X"][0] #first week of SKU 1 predictors

array([10.24,  9.86, 10.16,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ])

In [ ]:
data[1]['y'][0] #first week of SKU 1 value

110.0

In [ ]:
X_dict = dict()
y_dict = dict()

y_test = []
y_train = []

for sku in skus:
  X_train_sku, X_test_sku = np.split(data[sku]["X"], [int(0.7*len(data[sku]["X"]))]) #train-test splits
  y_train_sku, y_test_sku = np.split(data[sku]["y"], [int(0.7*len(data[sku]["y"]))])

  X_dict[sku] = {'train': X_train_sku, 'test': X_test_sku} #filling dictionaries
  y_dict[sku] = {'train': y_train_sku, 'test': y_test_sku}

  y_test += (list(y_test_sku)) #creating complete test/train arrays
  y_train += (list(y_train_sku))

# 2. Feature Selection and Regularization

### Ridge Model

In [ ]:
X_train_example, y_train_example = X_dict[1]['train'], y_dict[1]['train'] #example with only SKU 1
X_test_example, y_test_example = X_dict[1]['test'], y_dict[1]['test']

ridge_model = linear_model.Ridge(alpha = 1.0)
ridge_model.fit(X_train_example, y_train_example)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
ridge_df = pd.DataFrame(list(zip(colnames,ridge_model.coef_)),columns=["feature","coefficient"])
ridge_df[ridge_df['coefficient'] != 0]

,feature,coefficient
0,price,-2.323635
1,price-1,-0.735135
2,price-2,-1.048951
3,feat_main_page,19.285688
4,trend,1.226695
5,month_2,-7.191339
6,month_3,2.532871
7,month_4,2.115698
8,month_5,2.823665
9,month_6,3.758978


In [ ]:
ridge_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit a ridge regression model
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  ridge_model = linear_model.Ridge(alpha=1)
  ridge_model.fit(X_train, y_train)

  ridge_models[sku] = ridge_model
  y_test_pred += list(ridge_model.predict(X_test))

In [ ]:
len(y_test), len(y_test_pred)

(1320, 1320)

In [ ]:
print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Out of sample R2: 0.571
MSE: 47518.987


Trying to optimize the alpha selection at each step a bit more. Below is an example of the RidgeCV algorithm on one SKU.

In [ ]:
import random
X_train_example, y_train_example = X_dict[10]['train'], y_dict[10]['train'] #example with only SKU 1
X_test_example, y_test_example = X_dict[10]['test'], y_dict[10]['test']

ridge_model = linear_model.RidgeCV(alphas = [random.uniform(0, 2.5) for i in range(100)], scoring = 'r2') #RidgeCV takes a set of alphas, tries them all, and returns the best one using R2 as the metric
ridge_model.fit(X_train_example, y_train_example)

y_test_example_pred = ridge_model.predict(X_test_example)
print('Out of sample R2:',round(r2_score(y_test_example, np.array(y_test_example_pred)),3))
print('MSE:', round(mean_squared_error(y_test_example, np.array(y_test_example_pred)),3))
print('\n')

Out of sample R2: 0.035
MSE: 179.528




In [ ]:
ridge_model.alpha_ #optimal alpha value found above

0.2621409375277439

Decentralized Ridge Regression model with optimized hyperparameters for each SKU

In [ ]:
import time
start = time.time()
ridge_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit a ridge regression model AND FIND OPTIMAL ALPHA
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  ridge_model = linear_model.RidgeCV(alphas = [random.uniform(0, 2.5) for i in range(100)], scoring = 'r2')
  ridge_model.fit(X_train, y_train)

  ridge_models[sku] = ridge_model
  y_test_pred += list(ridge_model.predict(X_test))
print('Time to train:', round(time.time()-start, 2), 'seconds')

Time to train: 1.78 seconds


In [ ]:
print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Out of sample R2: 0.591
MSE: 45300.421


Trying to vary optimal hyperparameters in a slightly different way to monitor performance.

In [ ]:
alpha_values = dict()
for key in ridge_models:
  alpha_values[key] = [max(ridge_models[key].alpha_ + random.uniform(-0.1, 0.1), 0.00000001) for i in range(100)]

In [ ]:
import time
start = time.time()
ridge_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit a ridge regression model AND FIND OPTIMAL ALPHA
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  ridge_model = linear_model.RidgeCV(alphas = alpha_values[sku], scoring = 'r2')
  ridge_model.fit(X_train, y_train)

  ridge_models[sku] = ridge_model
  y_test_pred += list(ridge_model.predict(X_test))
print('Time to train:', round(time.time()-start, 2), 'seconds')

print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Time to train: 1.77 seconds
Out of sample R2: 0.59
MSE: 45455.758


Minimal difference here - it seems like we're about optimal in terms of alpha selection

The decentralized model which used all predictors had an MSE of 53,537.475, and an OOS R2 of 0.517, so this is a pretty considerable improvement.

### ElasticNet Model <br>
ElasticNetCV takes a list of l1 ratios to check, and autoomatically sets 100 alphas to determine where to compute the models. We vary only the l1 ratios here when optimizing performance as the alpha values are automatically calculated based on the l1 ratio inputs.

In [ ]:
# ElasticNetCV functions very similarly to the RidgeCV above
# we are trying to optimize l1_ratio for each observation

import time
start = time.time()
elasticnet_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit an ElasticNet regression model and find optimal l1 ratio
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  enet_model = linear_model.ElasticNetCV(l1_ratio = [0.01, .1, .3, .5, .7, .9, .95, .99, 1]) #based on sklearn docs recommended values
  enet_model.fit(X_train, y_train)

  elasticnet_models[sku] = enet_model
  y_test_pred += list(enet_model.predict(X_test))
print('Time to train:', round(time.time()-start, 2), 'seconds')

print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Time to train: 28.01 seconds
Out of sample R2: 0.573
MSE: 47331.758


Optimizing the ranges to look around for each SKU a bit more

In [ ]:
l1_ratios = dict()
for key in elasticnet_models:
  opt_ratio = elasticnet_models[key].l1_ratio_
  if opt_ratio == 0.01:
    l1_ratios[key] = [i*0.01 for i in range(1, 11)]
  elif opt_ratio == 0.1:
    l1_ratios[key] = [random.uniform(0.05, 0.25) for i in range(1, 11)]
  elif opt_ratio == 0.3:
    l1_ratios[key] = [random.uniform(0.2, 0.5) for i in range(1, 11)]
  elif opt_ratio == 0.5:
    l1_ratios[key] = [random.uniform(0.3, 0.7) for i in range(1, 11)]
  elif opt_ratio == 0.7:
    l1_ratios[key] = [random.uniform(0.5, 0.9) for i in range(1, 11)]
  elif opt_ratio == 0.9:
    l1_ratios[key] = [random.uniform(0.8, 0.95) for i in range(1, 11)]
  elif opt_ratio == 0.95:
    l1_ratios[key] = [random.uniform(0.9, 1) for i in range(1, 11)]
  elif opt_ratio == 0.99:
    l1_ratios[key] = [random.uniform(0.98, 1) for i in range(1, 11)]
  elif opt_ratio == 1:
    l1_ratios[key] = [1]

In [ ]:
import time
start = time.time()
elasticnet_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit an ElasticNet regression model and find optimal l1 ratio
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  enet_model = linear_model.ElasticNetCV(l1_ratio = l1_ratios[sku]) #based on sklearn docs recommended values
  enet_model.fit(X_train, y_train)

  elasticnet_models[sku] = enet_model
  y_test_pred += list(enet_model.predict(X_test))
print('Time to train:', round(time.time()-start, 2), 'seconds')

print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Time to train: 22.2 seconds
Out of sample R2: 0.573
MSE: 47279.903


Very marginal improvement - I would conjecture that we're about optimized in terms of l1 ratios.

This model is slightly less performant than the ridge regression one above, but still a significant improvement over the model which uses all variables

# 3. Log Transformations

In [ ]:
sales.head()

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_2,functionality_3,functionality_4,functionality_5,functionality_6,functionality_7,functionality_8,functionality_9,functionality_10,functionality_11,functionality_12,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,vendor_3,vendor_4,vendor_5,vendor_6,vendor_7,vendor_8,vendor_9,vendor_10
0,2016-11-14,1,110.0,10.24,9.86,10.16,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2016-11-21,1,127.0,8.27,10.24,9.86,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2016-11-28,1,84.0,8.83,8.27,10.24,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2016-12-05,1,87.0,8.98,8.83,8.27,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2016-12-12,1,64.0,10.40,8.98,8.83,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


The only columns which it would make sense to take the log transformation of would be the target, `weekly_sales`, or any of the price variables `price`, `price-1`, `price-2` and `trend`. The other variables are dummies, and would not benefit from a log transform.

### Log-Log transformation

In [ ]:
sales['weekly_sales'] = sales['weekly_sales'].mask(sales['weekly_sales'] == 0, 1) #replacing weeks with 0 sales with 1 - taking log of 0 yields -inf and creates errors
sales['trend'] += 1 #adding 1 to trend so that we do not take log of 0

In [ ]:
data = dict()
colnames = [col for col in sales.columns if col not in ["week", "weekly_sales", "sku"]] #variables we don't want as predictors
cols = [x for x in colnames if x not in ['price', 'price-1', 'price-2']] #taking log of all three price variables
for sku in skus:
  df_sku = sales[sales['sku'] == sku]
  data[sku] = {"X": np.concatenate((np.log(sales[sales['sku'] == sku][['price', 'price-1', 'price-2']]).values, sales[sales.sku == 1][cols].values), axis = 1), "y": np.log(df_sku["weekly_sales"].values)} #taking log of target

X_dict = dict()
y_dict = dict()

y_test = []
y_train = []

for sku in skus:
  X_train_sku, X_test_sku = np.split(data[sku]["X"], [int(0.7*len(data[sku]["X"]))]) #train-test splits
  y_train_sku, y_test_sku = np.split(data[sku]["y"], [int(0.7*len(data[sku]["y"]))])

  X_dict[sku] = {'train': X_train_sku, 'test': X_test_sku} #filling dictionaries
  y_dict[sku] = {'train': y_train_sku, 'test': y_test_sku}

  y_test += (list(y_test_sku)) #creating complete test/train arrays
  y_train += (list(y_train_sku))

Using ridge regression method from above to compare model performance (ridge regression was the most performant model thus far)

In [ ]:
import time
start = time.time()
ridge_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit a ridge regression model AND FIND OPTIMAL ALPHA
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  ridge_model = linear_model.RidgeCV(alphas = [random.uniform(0, 2.5) for i in range(100)], scoring = 'r2')
  ridge_model.fit(X_train, y_train)

  ridge_models[sku] = ridge_model
  y_test_pred += list(ridge_model.predict(X_test))
print('Time to train:', round(time.time()-start, 2), 'seconds')

print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Time to train: 1.74 seconds
Out of sample R2: 0.437
MSE: 0.716


This is quite a bit worse - let's check ElasticNet

In [ ]:
# ElasticNetCV functions very similarly to the RidgeCV above
# we are trying to optimize l1_ratio for each observation

import time
start = time.time()
elasticnet_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit an ElasticNet regression model and find optimal l1 ratio
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  enet_model = linear_model.ElasticNetCV(l1_ratio = [0.01, .1, .3, .5, .7, .9, .95, .99, 1]) #based on sklearn docs recommended values
  enet_model.fit(X_train, y_train)

  elasticnet_models[sku] = enet_model
  y_test_pred += list(enet_model.predict(X_test))
print('Time to train:', round(time.time()-start, 2), 'seconds')

print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Time to train: 32.22 seconds
Out of sample R2: 0.476
MSE: 0.666


Also not performing as well.

### Creating Function to Test <br>
Only testing ridge regression results initially because it's much faster than ElasticNet and they tend to move similarly

In [ ]:
def log_tests(predictors, target = False):
  data = dict()
  colnames = [col for col in sales.columns if col not in ["week", "weekly_sales", "sku"]] #variables we don't want as predictors
  cols = [x for x in colnames if x not in predictors] #taking log of all predictors passed in
  if target == True: # if we take the log of weekly_sales
    for sku in skus:
      df_sku = sales[sales['sku'] == sku]
      data[sku] = {"X": np.concatenate((np.log(sales[sales['sku'] == sku][predictors]).values, sales[sales.sku == 1][cols].values), axis = 1), "y": np.log(df_sku["weekly_sales"].values)} #taking log of target
  else:
    for sku in skus:
      df_sku = sales[sales['sku'] == sku]
      data[sku] = {"X": np.concatenate((np.log(sales[sales['sku'] == sku][predictors]).values, sales[sales.sku == 1][cols].values), axis = 1), "y": df_sku["weekly_sales"].values} #NOT taking log of target

  X_dict = dict()
  y_dict = dict()

  y_test = []
  y_train = []

  for sku in skus:
    X_train_sku, X_test_sku = np.split(data[sku]["X"], [int(0.7*len(data[sku]["X"]))]) #train-test splits
    y_train_sku, y_test_sku = np.split(data[sku]["y"], [int(0.7*len(data[sku]["y"]))])

    X_dict[sku] = {'train': X_train_sku, 'test': X_test_sku} #filling dictionaries
    y_dict[sku] = {'train': y_train_sku, 'test': y_test_sku}

    y_test += (list(y_test_sku)) #creating complete test/train arrays
    y_train += (list(y_train_sku))
  
  start = time.time()
  ridge_models = dict() #dictionary containing all the ridge models
  y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

  for sku in skus:
    #for each SKU, fit a ridge regression model AND FIND OPTIMAL ALPHA
    X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
    X_test = X_dict[sku]['test']

    ridge_model = linear_model.RidgeCV(alphas = [random.uniform(0, 2.5) for i in range(100)], scoring = 'r2')
    ridge_model.fit(X_train, y_train)

    ridge_models[sku] = ridge_model
    y_test_pred += list(ridge_model.predict(X_test))
  if target == True:
    print('Took log transform of: ' + str(predictors) + ', weekly_sales')
  else:
    print('Took log transform of: ' + str(predictors))
  print('Time to train:', round(time.time()-start, 2), 'seconds')
  print('Out of sample R2 (Ridge):',round(r2_score(y_test, np.array(y_test_pred)),3))
  print('MSE (Ridge):', round(mean_squared_error(y_test, np.array(y_test_pred)),3))
  print('\n')

In [ ]:
log_tests(predictors = ['price', 'price-1', 'price-2'], target = True)

Took log transform of: ['price', 'price-1', 'price-2'], weekly_sales
Time to train: 1.71 seconds
Out of sample R2 (Ridge): 0.443
MSE (Ridge): 0.709




Checking performance on every combination of the variables we might take a log transformation of.

In [ ]:
for combination in [['price']] + [['price-1']] + [['price-2']] + [['trend']] + [['price', 'price-1']] + [['price', 'price-2']] + [['price-1', 'price-2']] + [['price', 'trend']] + [['price-1', 'trend']] + [['price-2', 'trend']] + [['price', 'price-1', 'trend']] + [['price', 'price-2', 'trend']] + [['price-1', 'price-2', 'trend']] + [['price', 'price-1', 'price-2']] + [['price', 'price-1', 'price-2', 'trend']]:
  log_tests(predictors=combination, target=True)
  log_tests(predictors=combination, target=False)

Took log transform of: ['price'], weekly_sales
Time to train: 1.81 seconds
Out of sample R2 (Ridge): 0.392
MSE (Ridge): 0.774


Took log transform of: ['price']
Time to train: 1.74 seconds
Out of sample R2 (Ridge): 0.524
MSE (Ridge): 52671.416


Took log transform of: ['price-1'], weekly_sales
Time to train: 1.7 seconds
Out of sample R2 (Ridge): 0.232
MSE (Ridge): 0.978


Took log transform of: ['price-1']
Time to train: 1.76 seconds
Out of sample R2 (Ridge): 0.268
MSE (Ridge): 81029.076


Took log transform of: ['price-2'], weekly_sales
Time to train: 1.76 seconds
Out of sample R2 (Ridge): 0.188
MSE (Ridge): 1.033


Took log transform of: ['price-2']
Time to train: 1.71 seconds
Out of sample R2 (Ridge): 0.169
MSE (Ridge): 92006.488


Took log transform of: ['trend'], weekly_sales
Time to train: 1.75 seconds
Out of sample R2 (Ridge): 0.194
MSE (Ridge): 1.026


Took log transform of: ['trend']
Time to train: 1.69 seconds
Out of sample R2 (Ridge): 0.17
MSE (Ridge): 91925.077


Took log t

It looks like the best model is found by taking the log transform of each of the price variables, followed by taking the log transform of price and the one-week lagging price, and letting weekly sales remain as is.

### Log of `price`, `price-1`, and `price-2`

In [ ]:
data = dict()
colnames = [col for col in sales.columns if col not in ["week", "weekly_sales", "sku"]] #variables we don't want as predictors
cols = [x for x in colnames if x not in ['price', 'price-1', 'price-2']] #taking log of all three price variables
for sku in skus:
  df_sku = sales[sales['sku'] == sku]
  data[sku] = {"X": np.concatenate((np.log(sales[sales['sku'] == sku][['price', 'price-1', 'price-2']]).values, sales[sales.sku == 1][cols].values), axis = 1), "y": df_sku["weekly_sales"].values} #taking log of target

X_dict = dict()
y_dict = dict()

y_test = []
y_train = []

for sku in skus:
  X_train_sku, X_test_sku = np.split(data[sku]["X"], [int(0.7*len(data[sku]["X"]))]) #train-test splits
  y_train_sku, y_test_sku = np.split(data[sku]["y"], [int(0.7*len(data[sku]["y"]))])

  X_dict[sku] = {'train': X_train_sku, 'test': X_test_sku} #filling dictionaries
  y_dict[sku] = {'train': y_train_sku, 'test': y_test_sku}

  y_test += (list(y_test_sku)) #creating complete test/train arrays
  y_train += (list(y_train_sku))

In [ ]:
import time
start = time.time()
ridge_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit a ridge regression model AND FIND OPTIMAL ALPHA
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  ridge_model = linear_model.RidgeCV(alphas = [random.uniform(0, 2.5) for i in range(100)], scoring = 'r2')
  ridge_model.fit(X_train, y_train)

  ridge_models[sku] = ridge_model
  y_test_pred += list(ridge_model.predict(X_test))
print('Time to train:', round(time.time()-start, 2), 'seconds')

print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Time to train: 1.74 seconds
Out of sample R2: 0.576
MSE: 46964.75


In [ ]:
# ElasticNetCV functions very similarly to the RidgeCV above
# we are trying to optimize l1_ratio for each observation

import time
start = time.time()
elasticnet_models = dict() #dictionary containing all the ridge models
y_test_pred = [] #list containing the predictions by each ridge model for the weekly sales

for sku in skus:
  #for each SKU, fit an ElasticNet regression model and find optimal l1 ratio
  X_train, y_train = X_dict[sku]['train'], y_dict[sku]['train']
  X_test = X_dict[sku]['test']

  enet_model = linear_model.ElasticNetCV(l1_ratio = [0.01, .1, .3, .5, .7, .9, .95, .99, 1]) #based on sklearn docs recommended values
  enet_model.fit(X_train, y_train)

  elasticnet_models[sku] = enet_model
  y_test_pred += list(enet_model.predict(X_test))
print('Time to train:', round(time.time()-start, 2), 'seconds')

print('Out of sample R2:',round(r2_score(y_test, np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(y_test, np.array(y_test_pred)),3))

Time to train: 29.99 seconds
Out of sample R2: 0.562
MSE: 48484.724


It performs well, but slightly worse than taking no log transformations. We conclude that log transformations are most useful on the three price variables and not the target variable, but are still not preferable to taking no log transformations.

# 4. Fixed Effects Models

In [ ]:
sales_fe = sales.copy()
sales_fe.head()

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_2,functionality_3,functionality_4,functionality_5,functionality_6,functionality_7,functionality_8,functionality_9,functionality_10,functionality_11,functionality_12,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,vendor_3,vendor_4,vendor_5,vendor_6,vendor_7,vendor_8,vendor_9,vendor_10
0,2016-11-14,1,110.0,10.24,9.86,10.16,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2016-11-21,1,127.0,8.27,10.24,9.86,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2016-11-28,1,84.0,8.83,8.27,10.24,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2016-12-05,1,87.0,8.98,8.83,8.27,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2016-12-12,1,64.0,10.40,8.98,8.83,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
sales_fe = pd.get_dummies(sales_fe, columns=['sku'])
sales_fe.insert(1, 'sku', sales['sku'])
sales_fe.head()

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_2,functionality_3,functionality_4,functionality_5,functionality_6,functionality_7,functionality_8,functionality_9,functionality_10,functionality_11,functionality_12,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,...,sku_5,sku_6,sku_7,sku_8,sku_9,sku_10,sku_11,sku_12,sku_13,sku_14,sku_15,sku_16,sku_17,sku_18,sku_19,sku_20,sku_21,sku_22,sku_23,sku_24,sku_25,sku_26,sku_27,sku_28,sku_29,sku_30,sku_31,sku_32,sku_33,sku_34,sku_35,sku_36,sku_37,sku_38,sku_39,sku_40,sku_41,sku_42,sku_43,sku_44
0,2016-11-14,1,110.0,10.24,9.86,10.16,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2016-11-21,1,127.0,8.27,10.24,9.86,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016-11-28,1,84.0,8.83,8.27,10.24,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2016-12-05,1,87.0,8.98,8.83,8.27,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2016-12-12,1,64.0,10.40,8.98,8.83,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Price-fixed effects<br>
Each SKU has a different estimated price coefficient, but other coefficients are the same for all SKUs

For each SKU to have a different price coefficient, what we will do is create a dummy variable for each SKU such that the variable is 1 if we are observing that SKU and 0 otherwise (i.e., we will create variables `sku_1`, `sku_2`, ..., and `sku_n` will be equal to 1 if the row corresponds to data for SKU n, and 0 otherwise). Then, we will multiply these dummy variables by the price value. Finally we create a centralized model to predict `weekly_sales`. <br>
We multiply the price by the dummy variable so that each SKU's prices are given their own variable from which an estimated coefficient can be derived (because only the prices for that SKU are stored there, surrounded by 0's for all rows corresponding to other SKUs). All other variables will be calculated on an aggregate level, and thus be the same for all SKUs.

In [ ]:
sales_fe_price = sales_fe.copy()
sales_fe_price[['sku_'+str(i+1) for i in range(len(skus))]] = sales_fe_price[['sku_'+str(i+1) for i in range(len(skus))]].multiply(sales_fe_price[['price']].values) #pointwise multiplication of each dummy column by the price column
sales_fe_price.rename(columns = {'sku_'+str(sku): 'sku_'+str(sku)+'_price' for sku in skus}, inplace=True)

In [ ]:
sales_fe_price.tail()

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_2,functionality_3,functionality_4,functionality_5,functionality_6,functionality_7,functionality_8,functionality_9,functionality_10,functionality_11,functionality_12,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,...,sku_5_price,sku_6_price,sku_7_price,sku_8_price,sku_9_price,sku_10_price,sku_11_price,sku_12_price,sku_13_price,sku_14_price,sku_15_price,sku_16_price,sku_17_price,sku_18_price,sku_19_price,sku_20_price,sku_21_price,sku_22_price,sku_23_price,sku_24_price,sku_25_price,sku_26_price,sku_27_price,sku_28_price,sku_29_price,sku_30_price,sku_31_price,sku_32_price,sku_33_price,sku_34_price,sku_35_price,sku_36_price,sku_37_price,sku_38_price,sku_39_price,sku_40_price,sku_41_price,sku_42_price,sku_43_price,sku_44_price
4307,2018-08-27,44,20.0,53.99,42.38,43.99,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.99
4308,2018-09-03,44,14.0,52.99,53.99,42.38,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.99
4309,2018-09-10,44,22.0,44.99,52.99,53.99,1,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.99
4310,2018-09-17,44,28.0,42.99,44.99,52.99,1,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.99
4311,2018-09-24,44,26.0,43.45,42.99,44.99,1,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.45


In [ ]:
skus = list(set(sales_fe['sku']))
data = dict()
colnames = [col for col in sales_fe_price.columns if col not in ['week', 'weekly_sales', 'sku', 'price']] #excluding price from prediction since we don't want an aggregate price predictor variable

for sku in skus:
  df_sku = sales_fe_price[sales_fe_price['sku'] == sku]
  data[sku] = {'X': df_sku[colnames].values, 'y': df_sku['weekly_sales'].values}

In [ ]:
X_dict = {}
y_dict = {}

y_pred = []
y_test = []
y_train = []

train_size = 0
test_size = 0
range_dict = {}
row_train = 0
row_test = 0

for i in skus:
  
  X_train_i, X_test_i = np.split(data[i]["X"], [int(0.70 *98)]) #split for X
  y_train_i, y_test_i = np.split(data[i]["y"], [int(0.70 *98)]) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) #creating the complete training array
  y_train += list(y_train_i) #creating the complete testing array

  #Keep track of the position of the SKU in the dataset
  train_size += y_train_i.size
  test_size += y_test_i.size
  range_dict[i] = {'train': (row_train, row_train + y_train_i.size), 
                    'test':  (row_test, row_test + y_test_i.size) }
  row_train += y_train_i.size
  row_test += y_test_i.size

In [ ]:
row_train, len(y_train)

(2992, 2992)

In [ ]:
from sklearn.linear_model import LinearRegression
start_time = time.time()

X_central_train = X_dict[skus[0]]['train']
X_central_test = X_dict[skus[0]]['test']

for sku in skus[1:]:
  X_central_train = np.concatenate((X_central_train, X_dict[sku]['train']), axis = 0) #concatenate all the data row-wise
  X_central_test = np.concatenate((X_central_test, X_dict[sku]['test']), axis = 0) #concatenate all the data row-wise

model_central = LinearRegression().fit(X_central_train, y_train)
print("Out of Sample R2:", round(r2_score(y_test, model_central.predict(X_central_test)), 3))
print("MSE:", round(mean_squared_error(y_test, model_central.predict(X_central_test)), 3))
print("Time:", round(time.time() - start_time, 2), 'seconds')

Out of Sample R2: 0.367
MSE: 280307.9
Time: 0.06 seconds


Note that we use a linear regression here as opposed to ridge regression or ElasticNet. The latter options would almost certainly eliminate at least one of the SKU price coefficients, not allowing us to predict demand for some SKUs, then.

### Price and SKU Fixed Effects <br>
Each SKU has a different estimated price and intercept coefficient, but all other coefficients are the same for all SKUs.

The methodology will be similar to above. We create a column for each SKUs price, but additionally create a dummy variable column for each SKU. So, every row will have 44 price columns and 44 dummy columns (a price and dummy column correpsonding to each SKU).

In [ ]:
sales_fe_multi = sales_fe.copy()

In [ ]:
sales_fe_multi = sales_fe.copy()
sales_fe_multi[['sku_'+str(sku)+'_price' for sku in skus]] = sales_fe_multi[['sku_'+str(sku) for sku in skus]].multiply(sales_fe_multi[['price']].values) #pointwise multiplication of each dummy column by the price column
sales_fe_multi.head()

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_2,functionality_3,functionality_4,functionality_5,functionality_6,functionality_7,functionality_8,functionality_9,functionality_10,functionality_11,functionality_12,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,...,sku_5_price,sku_6_price,sku_7_price,sku_8_price,sku_9_price,sku_10_price,sku_11_price,sku_12_price,sku_13_price,sku_14_price,sku_15_price,sku_16_price,sku_17_price,sku_18_price,sku_19_price,sku_20_price,sku_21_price,sku_22_price,sku_23_price,sku_24_price,sku_25_price,sku_26_price,sku_27_price,sku_28_price,sku_29_price,sku_30_price,sku_31_price,sku_32_price,sku_33_price,sku_34_price,sku_35_price,sku_36_price,sku_37_price,sku_38_price,sku_39_price,sku_40_price,sku_41_price,sku_42_price,sku_43_price,sku_44_price
0,2016-11-14,1,110.0,10.24,9.86,10.16,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-11-21,1,127.0,8.27,10.24,9.86,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-11-28,1,84.0,8.83,8.27,10.24,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-12-05,1,87.0,8.98,8.83,8.27,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-12-12,1,64.0,10.40,8.98,8.83,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
sales_fe_multi[['sku_1', 'sku_1_price']] #checking to ensure that there are both dummy and price columns included
#dummy column will be the intercept coefficient for each SKU, and the price column will be the price coefficient for each SKU

,sku_1,sku_1_price
0,1,10.24
1,1,8.27
2,1,8.83
3,1,8.98
4,1,10.40
...,...,...
4307,0,0.00
4308,0,0.00
4309,0,0.00
4310,0,0.00


In [ ]:
skus = list(set(sales_fe['sku']))
data = dict()
colnames = [col for col in sales_fe_multi.columns if col not in ['week', 'weekly_sales', 'sku', 'price']] #excluding price from prediction since we don't want an aggregate price predictor variable

for sku in skus:
  df_sku = sales_fe_multi[sales_fe_price['sku'] == sku]
  data[sku] = {'X': df_sku[colnames].values, 'y': df_sku['weekly_sales'].values}

In [ ]:
X_dict = {}
y_dict = {}

y_pred = []
y_test = []
y_train = []

train_size = 0
test_size = 0
range_dict = {}
row_train = 0
row_test = 0

for i in skus:
  
  X_train_i, X_test_i = np.split(data[i]["X"], [int(0.70 *98)]) #split for X
  y_train_i, y_test_i = np.split(data[i]["y"], [int(0.70 *98)]) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) #creating the complete training array
  y_train += list(y_train_i) #creating the complete testing array

  #Keep track of the position of the SKU in the dataset
  train_size += y_train_i.size
  test_size += y_test_i.size
  range_dict[i] = {'train': (row_train, row_train + y_train_i.size), 
                    'test':  (row_test, row_test + y_test_i.size) }
  row_train += y_train_i.size
  row_test += y_test_i.size

In [ ]:
start_time = time.time()

X_central_train = X_dict[skus[0]]['train']
X_central_test = X_dict[skus[0]]['test']

for sku in skus[1:]:
  X_central_train = np.concatenate((X_central_train, X_dict[sku]['train']), axis = 0) #concatenate all the data row-wise
  X_central_test = np.concatenate((X_central_test, X_dict[sku]['test']), axis = 0) #concatenate all the data row-wise

model_central = LinearRegression().fit(X_central_train, y_train)
print("Out of Sample R2:", round(r2_score(y_test, model_central.predict(X_central_test)), 3))
print("MSE:", round(mean_squared_error(y_test, model_central.predict(X_central_test)), 3))
print("Time:", round(time.time() - start_time, 2), 'seconds')

Out of Sample R2: 0.58
MSE: 46528.648
Time: 0.06 seconds


This performs very well - much better than the fixed effects for SKU or prices alone models (0.298 and 0.367 R2 values, respectively), and MUCH better than the basic centralized approach (0.114 R2 value). It's comparable to the optimized ridge regression and ElasticNet models (best model there was ridge regression with no log transformations at 0.59 R2).